In [ ]:
from PIL import Image
from utils import SRC_DIR
from prompts.prompts import simple_reduce_prompt, medium_prompt, full_prompt

In [ ]:
image_path = SRC_DIR / 'babe/all-over-30-kourtney-kai-92423313/cropped/92423313_019_d0a6.jpg'
categories = [
    'armpit', 'ass', 'asshole', 'brunette', 'close up', 'feet', 'legs', 'lingerie', 'masturbation', 'mature',
    'milf', 'nipples', 'panties', 'pawg', 'pussy', 'solo', 'spreading', 'tattoo', 'tiny tits', 'undressing'
]
simple = simple_reduce_prompt(categories)
medium = medium_prompt(categories)
full = full_prompt(categories)

In [ ]:
Image.open(image_path).show()

In [ ]:
def benchmark(name: str):
    model_fn = None
    match name:
        case 'internvl':
            from VQA.internvl15.model import internvl15
            model_fn = internvl15
        case 'minicpm':
            from VQA.minicpm.model import minicpm_llama
            model_fn = minicpm_llama
        case 'composer':
            from VQA.intern_composer.model import composer
            model_fn = composer
        case 'miniintern':
            from VQA.mini_intern.model import miniintern
            model_fn = miniintern
        case 'blip':
            from VQA.blip.model import blip
            model_fn = blip

    simple_answer = model_fn(
        image_path=image_path,
        prompt=simple
    )
    medium_answer = model_fn(
        image_path=image_path,
        prompt=medium
    )
    full_answer = model_fn(
        image_path=image_path,
        prompt=full
    )

    with open(f"{name}.txt", 'w') as file:
        file.write(simple + "\n")
        file.write(simple_answer + "\n\n")
        file.write(medium + "\n")
        file.write(medium_answer + "\n\n")
        file.write(full + "\n")
        file.write(full_answer + "\n")

    print(f"Finished benchmark for {name}")



In [ ]:
from VQA.internvl15.model import internvl_15

intern_answer = internvl_15(
    image_path=image_path,
    prompt=simple
)

# RESULTS

In [1]:
# BLIP - Simple questions with a speed of 73 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)
# MININTERN - prblms
#

NameError: name 'BLIP' is not defined